<a href="https://colab.research.google.com/github/harshil0217/BERT_headline_classifier_v2/blob/main/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
os.chdir('drive/MyDrive/GitHub/BERT_headline_classifier_v2')

In [ ]:
from google.colab import userdata

username = userdata.get('githubusername')
password = userdata.get('githubpass')
email = userdata.get('githubemail')

!git config --global user.name "{username}"
!git config --global user.email "{email}"
!git config --global user.password "{password}"
!git config --global credential.helper store

In [ ]:
!git add .
!git commit -m "add training"
!git push https://{password}@github.com/harshil0217/BERT_headline_classifier_v2.git

[main a8565cf] add training
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite model_training.ipynb (90%)
fatal: could not read Password for 'https://{password}@github.com': No such device or address


In [ ]:
#import needed libraries

import pandas as pd
import numpy as np
import torch

from torch.utils.data.dataset import Dataset
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split


In [ ]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

,headline,dominant_emotion
0,India’s oldest e-commerce firm’s IPO is a rare...,pessimism
1,5-Year-Old Girl Forms Unbreakable Bond With A ...,other
2,India floods: Hundreds of passengers rescued f...,optimism
3,S&P 500 Hits Fresh Record High on Trade Optimism,other
4,3M stock price target cut to $195 from $205 at...,pessimism
...,...,...
1724,Review: ‘For the Good of the Game’,optimism
1725,Nasa’s Valkyrie robot could help build Mars base,optimism
1726,Bennet meets polling criterion for first Democ...,optimism
1727,Koch-funded group wants you to believe it’s sc...,pessimism
